In [ ]:
import os
import json
from zipfile import ZipFile
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50

In [ ]:
# Set Kaggle Credentials
kaggle_credentials = json.load(open("kaggle.json"))
os.environ['KAGGLE_USERNAME'] = kaggle_credentials["username"]
os.environ['KAGGLE_KEY'] = kaggle_credentials["key"]

In [ ]:
dataset_path = r"D:/project 7th sem/minor project/Dataset"
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset path not found: {dataset_path}")

In [ ]:
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")
if not os.path.exists(train_path) or not os.path.exists(valid_path):
    raise FileNotFoundError("Train or validation directories not found!")

In [ ]:
print(f"Number of categories in 'train': {len(os.listdir(train_path))}")
print(f"Sample categories in 'train': {os.listdir(train_path)[:5]}")
print(f"Number of categories in 'valid': {len(os.listdir(valid_path))}")
print(f"Sample categories in 'valid': {os.listdir(valid_path)[:5]}")

In [ ]:
# Image Parameters
img_size = 224
batch_size = 32

In [ ]:
# Data Augmentation for Training Data
data_gen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Validation Data Rescaling
data_gen_valid = ImageDataGenerator(rescale=1./255)

In [ ]:
# Data Generators
train_generator = data_gen_train.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = data_gen_valid.flow_from_directory(
    valid_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Load Pretrained Model
base_model = ResNet50(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

In [ ]:
# Build the Model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer='l2'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate the Model
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

In [ ]:
# Plot Training and Validation Metrics
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Evaluate the model on the validation data
y_true = validation_generator.classes  # True labels
y_pred = model.predict(validation_generator, steps=validation_generator.samples // batch_size)
y_pred_classes = np.argmax(y_pred, axis=1)  # Predicted labels

In [ ]:
model_save_path = 'D:/project 7th sem/minor project/model/crop_disease_prediction_model.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# Class Indices
class_indices = {v: k for k, v in train_generator.class_indices.items()}
json_path = os.path.join(dataset_path, 'class_indices.json')
json.dump(class_indices, open(json_path, 'w'))
print(f"Class indices saved to {json_path}")

In [ ]:
# Load and Preprocess Test Image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path)
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array.astype('float32') / 255.0
    return img_array

In [ ]:
# Predict the Class of an Image
def predict_image_class(model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name

In [ ]:
# Example Prediction
image_path = r"D:/project 7th sem/minor project/test/PotatoHealthy1.JPG"
if not os.path.exists(image_path):
    print(f"Test image not found at path: {image_path}")
else:
    predicted_class_name = predict_image_class(model, image_path, class_indices)
    print("Predicted Class Name:", predicted_class_name)